In [103]:
import io, pandas as pd, os
from PyPDF2 import PdfWriter, PdfReader
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from PIL import ImageFont

pdfmetrics.registerFont(TTFont('Montserrat-Bold', './FONTES/Montserrat-Bold.ttf'))
font = ImageFont.truetype('./FONTES/Montserrat-Bold.ttf', 13)

In [117]:
def cria_crachas(path_planilha, modelo):
    df = pd.read_excel(path_planilha)
    contagem = 1
    pagina = 1
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    t = df.columns[0].replace('/', '')
    local_saida = f'./SAIDA/CRACHAS_{t}.pdf'
    pos_inc = 0
    
    output = PdfWriter()
    for index, row in df.iterrows():
        nome = f'{row[0].upper().replace("MAYANNE", "MAYNNE")}'
        if (len(nome) < 18): 
            size = font.getsize(nome)[0]         
            pos_inc = abs(166 - size)/2
        elif (len(nome) > 18):
            nome = nome[0:18]
            size = font.getsize(nome)[0]           
            pos_inc = abs(166 - size)/2
        else:
            size = font.getsize(nome)[0]           
            pos_inc = abs(166 - size)/2

        if contagem == 1:
            packet = io.BytesIO()
            can = canvas.Canvas(packet, pagesize=A4)
            can.setPageRotation(90)
            can.setFillColorRGB(0.170, 0.140, 0.100)
            can.setFont("Montserrat-Bold", 13)
            can.drawString(108 + pos_inc, 365, nome)
        elif contagem == 2:
            can.drawString(339 + pos_inc, 365, nome)
        elif contagem == 3:
            can.drawString(570 + pos_inc, 365, nome)
        elif contagem == 4:
            can.drawString(108 + pos_inc, 78, nome)
        elif contagem == 5:
            can.drawString(339 + pos_inc, 78, nome)
        elif contagem == 6:
            can.drawString(570 + pos_inc, 78, nome)

        if (index == (len(df) - 1) or contagem == 6):
            can.save()
            packet.seek(0)
            new_pdf = PdfReader(packet)
            existing_pdf = PdfReader(open(modelo, "rb"))
            
            page = existing_pdf.pages[0]
            page.merge_page(new_pdf.pages[0])
            output.add_page(page)

            if (index < (len(df) - 1) and contagem == 6):
                contagem = 1
                pagina += 1
            else:
                outputStream = open(local_saida, "wb")
                output.write(outputStream)
                outputStream.close()
                pagina = 1
                contagem = 1
        else:
            contagem += 1

In [118]:
cria_crachas('./PLANILHAS/COMPRAS VERMELHO.xlsx', './MODELO/CRACHA - VERMELHO.pdf')
cria_crachas('./PLANILHAS/GERAL VERDE.xlsx', './MODELO/CRACHA - VERDE.pdf')
cria_crachas('./PLANILHAS/LANCHINHO CRACHA AMARELO.xlsx', './MODELO/CRACHA - AMARELO.pdf')
cria_crachas('./PLANILHAS/TRANSITO AMARELO.xlsx', './MODELO/CRACHA - AMARELO.pdf')

C:\Users\ricar\AppData\Local\Temp\ipykernel_11040\1048132050.py:15: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  size = font.getsize(nome)[0]
